<a href="https://colab.research.google.com/github/lrinakov/SA_project/blob/main/%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%BA%D0%BE%D0%B2%D0%B0%D0%BB%D0%B5%CC%88%D0%B2%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U flask-restful
!pip install -U pyngrok
!pip install dnspython
!pip install mongoengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.4 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok

ngrok_token = '2qZUVRO5jogTo4eDLGA2uUuzYEk_EBc85d3Y9Q6v56RHjcGa'
ngrok.set_auth_token(ngrok_token)
port = "5000"

In [3]:
import numpy as np
from mongoengine import *

In [4]:
mydb = 'kovaleva'

connect(host="mongodb+srv://cluster24.cfl8e.mongodb.net",
                 username = "hciuser2024",
                 password="VGLQAENYMKJFrjUX",
                 db = mydb)

MongoClient(host=['cluster24-shard-00-02.cfl8e.mongodb.net:27017', 'cluster24-shard-00-00.cfl8e.mongodb.net:27017', 'cluster24-shard-00-01.cfl8e.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-gcw9og-shard-0', tls=True, read_preference=Primary(), uuidrepresentation=3, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [5]:
from jinja2 import Template
from IPython.core.display import display, HTML

In [6]:
header = '''
<head>
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.5.0/font/bootstrap-icons.css">
</style>
</head>
'''
tmpl_pr = header + '''
<body>
<table class="table table-sm">
<thead>
  <tr>
    <th colspan="4" rowspan="4"><i class="bi bi-kanban" style="font-size: 7rem; color: cornflowerblue;"></i></th>
    <th colspan="4"><span style="font-weight:bold">Project name: </span>{{name}}</th>
  </tr>
  <tr>
    <th colspan="4"><span style="font-weight:bold">Difficulty: </span>{{difficulty}}</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td colspan="8"><span style="font-weight:bold">Status of execution:</span> {{status}}</td>
  </tr>
  <tr>
    <td colspan="8"><span style="font-weight:bold">Subtasks:</span> <br>
    {% for item in subtasks %}
  <li>{{ item }}</li>
{% endfor %}</td>
  </tr>
</tbody>
</table>
</body>
'''

In [7]:
tpr = Template(tmpl_pr)

In [8]:
tmpl_time = header + '''
<body>
<table class="table table-sm">
<tbody>
  <tr>
    <td colspan="8"><span style="font-weight:bold">Task name:</span> {{name}}</td>
  </tr>
  <tr>
    <td colspan="8"><span style="font-weight:bold">Time slots:</span> {{timeslots_availible}}</td>
  </tr>
</tbody>
</table>
</body>
'''

In [9]:
tprt = Template(tmpl_time)

In [10]:
import os
from flask import Flask, request, jsonify
from flask_restful import Resource, Api
from datetime import datetime
import requests

app = Flask(__name__)

ngrok_tunnel = ngrok.connect(port)
public_url = ngrok_tunnel.public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

app.config["BASE_URL"] = public_url
api = Api(app)

 * ngrok tunnel "https://2f87835cb840.ngrok-free.app" -> "http://127.0.0.1:5000"


In [11]:
class Fake(Document):
    p_id = IntField(required=True, unique=True)
    name = StringField(required=True)
    status = StringField(default="В Ожидании",choices=["В Ожидании","В Работе","Завершена"])
    subtasks = ListField()
    difficulty= IntField(default=0, min_value=0, max_value=10)
    date = DateField(default=datetime.now())
    deadline = DateField()
    physical_state = StringField(choices=["Отдохнувший", "Уставший", "Энергичный", "Сонный", "Нормальный"])
    mood = StringField(choices=["Хорошее", "Нейтральное", "Плохое"])
    stress_level = IntField(min_value=0, max_value=10)

In [12]:
ProjectA = Fake(p_id=2345780,name= 'App development',
         subtasks=['User research', 'Requirement creation', 'Development', 'Usability testing'],
         difficulty= 7, date= "2025-06-18", deadline="2025-10-28").save()

ProjectB = Fake(p_id=23456890,name= 'HR report',
         subtasks=['Statistics concolidation', 'Dashboard prerparation', 'Presentation'],
         status='Завершена', difficulty = 4, date= "2025-06-18", deadline="2025-10-18").save()

NotUniqueError: Tried to save duplicate unique keys (E11000 duplicate key error collection: kovaleva.fake index: p_id_1 dup key: { p_id: 2345780 }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: kovaleva.fake index: p_id_1 dup key: { p_id: 2345780 }', 'keyPattern': {'p_id': 1}, 'keyValue': {'p_id': 2345780}})

In [13]:
@app.route("/")
def home():
    return "<h3>You have entered the main page. You are authorized as user.</h3> <br> <h2>Enjoy your work!! </h2>"

@app.route('/project/<pr_id>', methods=['GET'])
def get_project(pr_id):
    project = Fake.objects.get(p_id=pr_id)
    project_data = {
            'name': project.name,
            'status': project.status,
            'subtasks': project.subtasks,
            'difficulty': project.difficulty,
            'date': project.date,
            'deadline': project.deadline,
            }
    return Template(tmpl_pr).render(project_data)

@app.route('/project', methods=['POST'])
def create_project():
    data = request.get_json()
    if not data or 'p_id' not in data or 'name' not in data:
        return jsonify({'error': 'p_id и name обязательны'}), 400
    if Fake.objects(p_id=data['p_id']).first():
        return jsonify({'error': 'Задача с таким p_id уже существует'}), 400
    project = Fake(
        p_id=data['p_id'],
        name=data['name'],
        status=data.get('status', 'Pending'),
        subtasks=data.get('subtasks', []),
        difficulty=data.get('difficulty', 0),
        date=data.get('date', datetime.now()),
        deadline=data.get('deadline')
    )
    project.save()
    return jsonify({'message': 'Задача успешно создана', 'p_id': project.p_id}), 201

@app.route('/project/<int:pr_id>/complete', methods=['PATCH'])
def complete_project(pr_id):
    project = Fake.objects(p_id=pr_id).first()
    if not project:
        return jsonify({'error': 'Задача не найдена'}), 404

    data = request.get_json()
    if not data:
        return jsonify({'error': 'Ожидается JSON с данными состояния пользователя'}), 400

    physical_state = data.get('physical_state')
    mood = data.get('mood')
    stress_level = data.get('stress_level')

    if stress_level is not None and (not isinstance(stress_level, int) or stress_level < 0 or stress_level > 10):
        return jsonify({'error': 'Поле stress_level должно быть целым числом от 0 до 10'}), 400

    project.status = "Завершена"
    if physical_state:
        project.physical_state = physical_state
    if mood:
        project.mood = mood
    if stress_level is not None:
        project.stress_level = stress_level

    project.save()
    return jsonify({
        'p_id': project.p_id,
        'status': project.status,
        'user_feedback': {
            'physical_state': project.physical_state,
            'mood': project.mood,
            'stress_level': project.stress_level
        }
    }), 200

@app.route('/analytics', methods=['GET'])
def get_analytics():
    try:
        end_date = datetime.now().date()
        start_date = end_date - timedelta(days=6)

        start_str = start_date.strftime("%Y-%m-%d")
        end_str = end_date.strftime("%Y-%m-%d")

        daily_load = []
        for i in range(7):
            day = start_date + timedelta(days=i)
            daily_load.append({
                'date': day.strftime("%Y-%m-%d"),
                'tasks_completed': random.randint(0, 5),
                'average_difficulty': random.randint(1, 5)
            })

        insights = {
            'most_productive_day': max(daily_load, key=lambda x: x['tasks_completed'])['date'],
            'avg_difficulty': round(sum(d['average_difficulty'] for d in daily_load) / len(daily_load), 2),
            'avg_tasks_per_day': round(sum(d['tasks_completed'] for d in daily_load) / len(daily_load), 2),
            'user_state': random.choice(['В норме', 'Перегружен', 'Нужен отдых'])
        }

        return jsonify({
            'period': {'start': start_str, 'end': end_str},
            'daily_load': daily_load,
            'insights': insights,
            'diagram': 'В будущем здесь появится автоматически генерируемая визуализация аналитики.'
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/recommendations', methods=['GET'])
def get_recommendations():
    return jsonify({
        'recommendations': [
        {
            'category': 'Нагрузка',
            'advice': 'Уменьшите количество задач с высоким уровнем сложности в первой половине недели.'
        },
        {
            'category': 'Отдых',
            'advice': 'Добавьте 1 день без задач после особенно загруженных дней.'
        },
        {
            'category': 'Оптимизация',
            'advice': 'Сгруппируйте похожие задачи по типу и выполняйте их блоками для повышения фокуса.'
        },
        {
            'category': 'Продуктивность',
            'advice': 'Планируйте сложные задачи утром, когда уровень концентрации выше.'
        }
    ]
    })

In [14]:
import json
json_data2 = Fake.objects.to_json()
dicts = json.loads(json_data2)
dicts

[{'_id': {'$oid': '6769384de02dc647d61cebc3'},
  'p_id': 13349,
  'name': 'App development',
  'nppl': 23,
  'department': 'IT',
  'status': 'Pending',
  'plan': ['User research',
   'Requirement creation',
   'Development',
   'Usability testing']},
 {'_id': {'$oid': '67693a0be02dc647d61cebca'},
  'p_id': 1334910,
  'name': 'App development',
  'nppl': 23,
  'department': 'IT',
  'status': 'Pending',
  'plan': ['User research',
   'Requirement creation',
   'Development',
   'Usability testing']},
 {'_id': {'$oid': '676948f9e02dc647d61cebce'},
  'p_id': 1348910,
  'name': 'HR report',
  'nppl': 8,
  'department': 'R&D',
  'status': 'Done',
  'plan': ['Statistics concolidation',
   'Dashboard prerparation',
   'Presentation']},
 {'_id': {'$oid': '67694928e02dc647d61cebd3'},
  'p_id': 31,
  'name': 'Branch opening',
  'nppl': 117,
  'department': 'R&D',
  'status': 'Pending',
  'plan': ['Research', 'Budjeting', 'Staffing', 'Opening day celebration']},
 {'_id': {'$oid': '6769515ee02dc647

In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Oct/2025 13:17:42] "GET / HTTP/1.1" 200 -


In [ ]:
disconnect()